# Applying l-diversity to data minimization to a trained regression ML model

In this tutorial I will show how to use the l_diversity.py and privacy_grid.py modules from the new apt/security package.

These are meant to be enhancements to the minimization.py framework explored by the file goldsteen2022data.pdf

## Setup

In [1]:
!git clone https://github.com/cmalvr/ai-privacy-toolkit.git

Cloning into 'ai-privacy-toolkit'...
remote: Enumerating objects: 1528, done.
remote: Counting objects: 100% (595/595), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 1528 (delta 490), reused 375 (delta 370), pack-reused 933 (from 2)
Receiving objects: 100% (1528/1528), 1.70 MiB | 6.53 MiB/s, done.
Resolving deltas: 100% (1035/1035), done.


In [2]:
! python --version

Python 3.11.11


In [3]:
%cd /content/ai-privacy-toolkit/
import sys
sys.path.append('/content/ai-privacy-toolkit/apt')
print(sys.path)

/content/ai-privacy-toolkit
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/ai-privacy-toolkit/apt']


In [4]:
! pip install -r /content/ai-privacy-toolkit/requirements.txt;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta

In [5]:
! pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.4 MB/s eta 0:00:00


## Load data
QI parameter for Quasi Indentifier deifinition.

In [6]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

dataset = load_breast_cancer()
X, y = dataset.data, dataset.target
features = list(dataset.feature_names)  # All features from the dataset
QI = ["mean texture", "mean perimeter", "mean smoothness"]
sensitive_attribute = "mean radius"

In [7]:
print(features)

['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


## Train DecisionTreeRegressor model

In [8]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeRegressor

#Dataset split to train baseline model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=14)

# Train a DecisionTreeRegressor as our baseline model.
model = DecisionTreeRegressor(random_state=10, min_samples_split=2)
model.fit(X_train, y_train)

#Measure base line accuracy
print('Base model accuracy (R2 score): ', model.score(X_test, y_test))

Base model accuracy (R2 score):  0.579424475598187


## Data pre-processing


In [9]:
from apt.utils.datasets import ArrayDataset

# In a real-world scenario, the model is pre-trained and we no longer have access to the original training data.
# Here, we assume the test set is all we have.
# Create a DataFrame for the test data.
df_test = pd.DataFrame(X_test, columns=features)
df_test['target'] = y_test

# Wrap the test data into an ArrayDataset.
test_dataset = ArrayDataset(df_test.drop(columns=['target']), df_test['target'])
test_dataset.features_names = list(df_test.columns.drop('target'))

## l-diversity and grid-search: Application



In [10]:
from apt.security.privacy_grid import grid_search_privacy
from apt.security.privacy_grid import display_grid_search_results
import contextlib, io


# Set the desired target accuracy for the minimizer.
target_accuracy = 0.6

#Grid Search (Silencing the minimizer)
with contextlib.redirect_stdout(io.StringIO()):
  results = grid_search_privacy(
      dataset=test_dataset,
      sensitive_attribute="mean radius",
      quasi_identifiers=["mean texture", "mean perimeter", "mean smoothness"],
      model=model,
      features=features,
      target_accuracy=target_accuracy,
      k_min=2, k_max=100,   # Adjust the range as needed.
      l_min=1
  )

In [11]:
#Example display of the results
display_grid_search_results(results, min_accuracy=0.7, sort_by="deletion_ratio")

Parameters: k=83, l=79
Deletion Ratio: 0.291
Accuracy on minimized data: 0.700
Generalizations: {'ranges': {'mean texture': [], 'mean perimeter': [91.09000015258789], 'mean smoothness': []}, 'categories': {}, 'untouched': ['worst texture', 'mean compactness', 'fractal dimension error', 'worst area', 'mean concave points', 'worst fractal dimension', 'worst perimeter', 'worst smoothness', 'perimeter error', 'mean fractal dimension', 'compactness error', 'concave points error', 'radius error', 'worst concavity', 'worst symmetry', 'worst concave points', 'area error', 'mean area', 'concavity error', 'mean concavity', 'worst radius', 'mean symmetry', 'symmetry error', 'mean radius', 'texture error', 'worst compactness', 'smoothness error'], 'category_representatives': {}, 'range_representatives': {'mean texture': [], 'mean perimeter': [91.09000015258789], 'mean smoothness': []}}
----------------------------------------
Parameters: k=83, l=80
Deletion Ratio: 0.291
Accuracy on minimized data: